Scraping Github Repos from multiple pages

In [3]:
import requests
import pandas as pd
import time

In [22]:
project_name = []
last_update = []

In [27]:
def get_projects(stars_range):
    i = 1;
    while i <= 3:
    
        headers = {
            'authority': 'github.com',
            'accept': 'application/json',
            'accept-language': 'en-US,en;q=0.9',
            'referer': 'https://github.com/search?q=stars%3A{stars_range}&type=repositories&p=1',
            'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
            'x-github-target': 'dotcom',
            'x-requested-with': 'XMLHttpRequest',
        }
        
        params = {
            'q': 'stars:{stars_range}',
            'type': 'repositories',
            'p': str(i),
        }
        
        try:
            # response variable
            response = requests.get('https://github.com/search', params=params, headers=headers) 
            response.raise_for_status()
            print(f"Page {i} - Status Code: {response.status_code}")
            
            # json object
            results_json = response.json()
        except requests.exceptions.RequestException as e:
            print(f"Request failed on page {i}: {e}")
            delay_seconds = 60  # default delay
            time.sleep(delay_seconds)
            continue
            
        except json.decoder.JSONDecodeError as e:
            print(f"JSONDecodeError on page {i}: {e}")
        
        project_list = results_json['payload']['results']
        
        for project in project_list:
            project_name.append(project['hl_name'])
            last_update.append(project['repo']['repository']['updated_at'])
        
        i += 1


In [29]:
get_projects('90000..100000')

Page 1 - Status Code: 200
Page 2 - Status Code: 200
Page 3 - Status Code: 200


In [30]:
projects_df = pd.DataFrame({'Project':project_name, 'Last Update':last_update})

In [31]:
projects_df

,Project,Last Update
0,f/awesome-chatgpt-prompts,2024-02-09T02:04:13.449Z
1,Genymobile/scrcpy,2024-02-16T10:59:55.295Z
2,mrdoob/three.js,2024-02-16T17:35:45.432Z
3,microsoft/TypeScript,2024-02-16T18:36:31.193Z
4,goldbergyoni/nodebestpractices,2024-02-11T22:58:52.398Z
5,angular/angular,2024-02-16T17:37:52.593Z
6,microsoft/terminal,2024-02-16T11:49:47.412Z
7,denoland/deno,2024-02-16T18:23:43.889Z
8,mui/material-ui,2024-02-16T16:43:09.852Z
9,rust-lang/rust,2024-02-16T18:33:40.764Z


In [7]:
#projects_df.to_excel("projects_test.xlsx", index=False)

try:
    with pd.ExcelWriter(
        "projects_9500AndUp.xlsx",
        mode="a",
        engine="openpyxl",
        if_sheet_exists="overlay",
    ) as writer:
         projects_df.to_excel(writer,sheet_name="Sheet1", startrow=writer.sheets["Sheet1"].max_row, index = False,header= False)
except FileNotFoundError:
    projects_df.to_excel("projects_9500AndUp.xlsx", index=False)